In [1]:
from delta import configure_spark_with_delta_pip, DeltaTable
from pyspark.sql import SparkSession

In [2]:
builder = (SparkSession.builder
           .appName("manage-delta-table")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-590762c8-e229-481c-b0cc-433a4d232e25;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 325ms :: artifacts dl 15ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0 

In [3]:
%load_ext sparksql_magic
%config SparkSql.limit=20

In [4]:
df = spark.read.format("delta").load("/opt/workspace/data/delta_lake/netflix_titles")
df.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                null|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|               |Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         null|Septem

### Table Constraints

In [5]:
%%sparksql
ALTER TABLE delta.`/opt/workspace/data/delta_lake/netflix_titles` ALTER COLUMN title DROP NOT NULL

In [6]:
%%sparksql
ALTER TABLE delta.`/opt/workspace/data/delta_lake/netflix_titles` ADD CONSTRAINT validType CHECK (type IN ('Movie', 'Show','TV Show'));

### Cloning Delta Tables

In [7]:
%%sparksql
CREATE OR REPLACE TABLE delta.`/opt/workspace/data/delta_lake/netflix_titles_shallow_clone` SHALLOW CLONE delta.`/opt/workspace/data/delta_lake/netflix_titles`

source_table_size,source_num_of_files,num_removed_files,num_copied_files,removed_files_size,copied_files_size
1963749,1,0,0,0,0


### Update Schema

In [8]:
%%sparksql
ALTER TABLE delta.`/opt/workspace/data/delta_lake/netflix_titles` ADD COLUMNS (ID INT)

In [9]:
%%sparksql
ALTER TABLE delta.`/opt/workspace/data/delta_lake/netflix_titles` ALTER COLUMN country AFTER show_id;

In [10]:
%%sparksql
ALTER TABLE delta.`/opt/workspace/data/delta_lake/netflix_titles` ALTER COLUMN release_year AFTER show_id;

In [11]:
%%sparksql
ALTER TABLE delta.`/opt/workspace/data/delta_lake/netflix_titles` SET TBLPROPERTIES (
    'delta.minReaderVersion' = '2',
    'delta.minWriterVersion' = '5',
    'delta.columnMapping.mode' = 'name'
  )

In [12]:
%%sparksql
ALTER TABLE delta.`/opt/workspace/data/delta_lake/netflix_titles` RENAME COLUMN listed_in TO genres

In [13]:
%%sparksql
ALTER TABLE delta.`/opt/workspace/data/delta_lake/netflix_titles` DROP COLUMN ID

In [14]:
spark.stop()